In [25]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

import implicit
import lightfm
import warnings
warnings.filterwarnings('ignore')

In [26]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [27]:
actions = pd.read_pickle(PREPARED_PATH+'actions_one_table.pkl')

In [28]:
actions.sort_index(inplace = True) # На всякий случай, иначе деление не будет работать

In [29]:
actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [30]:
def get_train_test(actions,mode = 'by_time',perc = (0.6,0.2,0.2)):
    '''
    здесь не очень аккуратно обращаемся с временем просмотра, потому что фильмы на границе должны быть 
    с обрезанной длительностью - но насрать
    '''
    X = actions.copy()
    if mode == 'by_time':
        X['ones'] = 1
        X['increment'] = np.arange(len(X))
        by_time = X.groupby(level = 2)['ones'].sum()
        by_time.sort_index(inplace = True)
        #проверили, что вроде как все ок и равномерно во времени
        cur = 0
        idx = []
        for i in range(len(perc)):
#             print(np.round((cur)*len(by_time)),np.round((cur+perc[i])*len(by_time)))
            by_time_temp = by_time.iloc[int(np.round((cur)*len(by_time))):int(np.round((cur+perc[i])*len(by_time)))].index.values
            print(len(by_time_temp))
            mn = by_time_temp.min()
            mx = by_time_temp.max()
            cur+=perc[i]
            idx.append(X.loc[(slice(None),slice(None),slice(mn,mx)),'increment'].values)
            
        return idx

In [31]:
idx = get_train_test(actions)

6558458
2186152
2186153


In [32]:
actions.iloc[idx[0]].index.get_level_values(2).max()

43362401.96226887

In [33]:
actions.iloc[idx[1]].index.get_level_values(2).min()

43362401.97085199

In [34]:
actions.iloc[idx[1]].index.get_level_values(2).max()

43828341.47903843

In [35]:
actions.iloc[idx[2]].index.get_level_values(2).min()

43828341.48519237

In [36]:
actions.consumption_mode.value_counts()

S    8296227
P     873834
R     472951
Name: consumption_mode, dtype: int64

In [37]:
actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [38]:
# Вроде не пересекается.
train,test,valid = actions.iloc[idx[0]],actions.iloc[idx[1]],actions.iloc[idx[2]]

In [39]:
def get_target(actions):
    '''
    Функция, которая вернет число просмотреннх серий каждым пользователем каждого сериала, потом вернет то,что недопотребил
    А потом то, что точно потребил согласно правилам соревнования - например, так можно вычислить примерную длительность сериала 
    и его же рекомендовать в потребленные после.
    '''
    watch_actions = actions[actions.action == 'watch']
    # Блок нахождения всяких статистик по сериалам
    serials = watch_actions[watch_actions['type'] != 1]
    # Заменим длиетльность на 0, там где длительности нет.. или это очень короткие, надо подумоть.
    serials['num_of_series'] = (serials['watched_time']/serials['duration']).fillna(0).replace(np.inf,0).astype(int)
    serials['time_being'] = serials.index.get_level_values(2)
    dur_being = serials.groupby(level = 1).agg({'time_being':[min,len],'num_of_series':[lambda x:x.mode()[0],max]})
    dur_being.columns = ['time_being','count_of_watch','num_of_series_mode','num_of_series_max']
    
    # Блок нахождения статистик по фильмам для пользователя
    films = watch_actions[watch_actions['type'] == 1]
    # Здесь важно видимо, как долго смотрел
    films['time_being'] = films.index.get_level_values(2)
    dur_films = films.groupby(level = 1).agg({'time_being':[min,len]})
    dur_films.columns = ['time_being','count_of_watch']
    
    # Блок нахождения статистик по фильмам и пользователям
    watch_actions['rel_dur'] = (watch_actions['watched_time']/watch_actions['duration'])
    target = 1*(watch_actions['rel_dur'] >= 1/3) | watch_actions['consumption_mode'].isin(['R','P']) 
    target = target.groupby(level = [0,1]).mean()
    watch_actions = watch_actions.groupby(level = [0,1]).mean()
    watch_actions['rel_dur'] = watch_actions['rel_dur'].replace(np.inf,1)# Заглушка для фильмов с 0 длительностью
    
    
    
    return dur_being,dur_films,watch_actions,target

In [40]:
%time 
dur_being_train,dur_films_train,watch_actions_train,target_train = get_target(train)
dur_being_test,dur_films_test,watch_actions_test,target_test = get_target(test)
dur_being_valid,dur_films_valid,watch_actions_valid,target_valid = get_target(valid)

Wall time: 0 ns


In [41]:
# Получили фичи для фильмов
import pickle
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './prepared_data/catalogue_features.pkl'

In [ ]:
# Получили фичи для юзеров (пока какие-то)
# import pickle
# with open(PREPARED_PATH+'catalogue_users.pkl','rb') as f:
#     match_element_row_user,match_row_element_user,match_columns_user,element_matrix_user = pickle.load(f)
# Кажется, что их правильно пересчитывать по тем, кто есть во времени сейчас. 
# Фильмы вроде как мы занем все и не знаем про новые и это не страшно - Про это возможно стоит подумать
import pickle
from sklearn.feature_extraction.text import CountVectorizer

with open(PREPARED_PATH+'bag_of_attr_movie.pkl','rb') as f:
    bag_of_attr = pickle.load(f)
def get_users_features(actions,bag_of_attr):
    '''
    bag_of_attr - словарь, где просто каждому id  фильма сопоставлена строка атрибутов через запятую.
    строго  говоря в просмотренных фильмах атрибутов может оказаться меньше, чем во всем пуле фильмов, но я 
    пока не знаю проблема ли это ToDo
    '''
    # Приделаем каждому чуваку атрибуты просмотренных фильмов. ну или вообще по всем действиям - они все позитивные
    ind_user = []
    buf = []
    for i in tqdm.tqdm(np.unique(actions.index.get_level_values(0))):
        temp = np.unique(actions.loc[i].index.get_level_values(0))
        ind_user.append(i)

        s = ''
        for ii in temp:
            s+=bag_of_attr[ii]

            s+=','
        #assert X.shape[1] == len(a)
        buf.append(s)

    cv1 = CountVectorizer(token_pattern='\d+',)
    X_user = cv1.fit_transform(buf)
    match_user_row = {i:ii for ii,i in enumerate(ind_user)}
    match_row_user = {ii:i for ii,i in enumerate(ind_user)}
    match_columns_user = list(cv1.get_feature_names())
    
    return match_user_row,match_row_user,match_columns_user,X_user

In [ ]:
from scipy.sparse import coo_matrix
def df_to_matrix(X,match_user_row,match_element_row):
    '''
    На вход подается датафрейм с мультииндексом <user_id, element_id> и некоторой оценкой пары, затем он переупорядочивается и дополняется 
    по шаблонам из строк всяких спарс матричек для фильмов и юзеров
    match_user_row - отображение из айди в номер строки в матрице, match_element_row - аналогично
    '''
    Y = X.copy()
    Y['users'] = Y.index.get_level_values(0).map(match_user_row).astype(int)
    Y['items'] = Y.index.get_level_values(1).map(match_element_row).astype(int)
    print(Y['users'].values.max())
    return coo_matrix((X.values.squeeze(),(Y['users'].values,Y['items'].values)))

In [ ]:
# Получим что-то сначала для трейна, причем для рейтингового
# Вообще парллелится, но пока непонятно зачем кроме тренировки
match_user_row,match_row_user,match_columns_user,train_user = get_users_features(train[train.action =='rate'],bag_of_attr)

In [ ]:
X = train.loc[train.action =='rate','rating'].groupby(level = [0,1]).mean().to_frame()
# X.value_counts()
train_matrix = df_to_matrix(X,match_user_row,match_element_row)

In [ ]:
print(len(match_user_row),len(match_element_row),train_matrix.shape)

In [ ]:
# def fit_lightfm(train,item_features=None,seed = 0)
seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)


model.fit(train_matrix,user_features = train_user,item_features = element_matrix,epochs = epochs,num_threads = num_threads,
         verbose = True)

In [ ]:
def get_answer(test_users_dict,test):
    pass